In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras import layers, models

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)
# 자주쓰이는 단어 20000개 까지만 사용을 하고, 나머지는 [unk] (여기 코드는 0 으로 바꿈)


17464789/17464789 [==============================] - 0s 0us/step


In [9]:
import numpy as np
x_train.shape
# 리뷰(데이터) 의 길이가  다 다르다! 
# 길이를 몇 정도로 통일하는게 좋을지 가설을 내려보자

## 리뷰 길이 구하기
len_review = []
for sentence in x_train:
    len_review.append(len(sentence))

#len_review = [len(review) for review in x_train]
##


리뷰의 최댓값 2494


In [14]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,

In [11]:
print(f'리뷰의 최댓값 {np.max(len_review)}')
print(f'리뷰의 평균 {np.mean(len_review)}')

리뷰의 최댓값 2494
리뷰의 평균 238.71364


In [ ]:
import matplotlib.pyplot as plt

plt.hist(len_review,bins=50)
# 길이 를 238로 잡자!


In [ ]:
a = [1,2,3,4,5] 
# 3 배

x = [[1,2,4,5]  [1 2 3 5]]
y = [[3] , [4]]

layer.dense(100,input_shape=(4,))
layer.dense(1)